In [1]:
import os
import json
import pandas as pd
import numpy as np
from pytorch_transformers import BertModel,BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from tqdm import tqdm, trange



In [2]:
!nvidia-smi

/bin/sh: nvidia-smi: command not found


In [3]:
from my_process_json import *

In [82]:
#data_reader = MyDataReader('/home/akobtan/NLP_Project/Project/clicr_dataset/train1.0.json',bs=10000)
data_reader = MyDataReader(bs=500)

In [83]:
mydata = data_reader.send_batches()

In [84]:
dataset_size = data_reader.get_data_size()
dataset_size

500

In [149]:
paragraphs = [e['p'] for e in mydata].copy()
paragraph_tags = [e['p_tags'] for e in mydata].copy()
queries = [e['q'] for e in mydata].copy()
query_tags = [e['q_tags'] for e in mydata].copy()

In [150]:
#creating embeddings for a batch sample from CLICR
tags_vals = ['B-ans','I-ans','O']
tag2idx = {t: i for i, t in enumerate(tags_vals)}
tag2idx['[PAD]'] = -100
tags_vals+= ['[PAD]']
tag2idx['[SEP]'] = -100
tags_vals+= ['[SEP]']
tag2idx['[CLS]'] = -100
tags_vals+= ['[CLS]']

In [142]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [11]:
device

device(type='cpu')

In [ ]:
torch.cuda.get_device_name(0) 

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [138]:
print(len(p_tags[1]))

1025


In [137]:
len(paragraphs[1].split())

1025

In [151]:
import copy
split_size = 200
p_tags = copy.deepcopy(paragraph_tags)
q_tags = copy.deepcopy(query_tags)
tokenized_texts = []
labels = []
for i,paragraph in enumerate(paragraphs):
    texts = []
    paragraph_tag = copy.deepcopy(p_tags[i])
    #number of splits for current paragraph
    n_splits = int(len(paragraph.split())/split_size)
    #tokenize split paragraph text appended by full query at the end with <sep> token between paragraph and query
    queries[i] = queries[i].replace("@placeholder","B")
    texts += [s for s in ['[CLS] ' + ' '.join(paragraph.split()[(split_size*split):(split_size*(split+1))]) + ' [SEP] ' + queries[i] + ' [SEP]' for split in range(n_splits)]]
    tokenized_texts += [tokenizer.tokenize(s) for s in ['[CLS] ' + ' '.join(paragraph.split()[(split_size*split):(split_size*(split+1))]) + ' [SEP] ' + queries[i] + ' [SEP]' for split in range(n_splits)]]
    #tokenize remainder of splits
    if int(len(paragraph.split()) % split_size) > 0:
        texts += ['[CLS] ' + ' '.join(paragraph.split()[(n_splits*split_size):]) + ' [SEP] ' + queries[i] + ' [SEP]']
        tokenized_texts += [tokenizer.tokenize('[CLS] ' + ' '.join(paragraph.split()[(n_splits*split_size):]) + ' [SEP] ' + queries[i] + ' [SEP]')]
    
    #create labels to be used for tagging from the text
    for split_index, sent in enumerate(texts):
        l = []
        for word_index,word in enumerate(sent.split()):
            if '[SEP]' in word:
                l += ['[SEP]']
                break
            if '[CLS]' in word:
                l += ['[CLS]']
                continue
            if not paragraph_tag:
                print(sent,'\n')
                print(word)
                print(split_index)
            lab = paragraph_tag.pop(0)
            word_list = tokenizer.tokenize(word)
            for w in word_list:
                l += [lab]
        query_tags = copy.deepcopy(q_tags[i])
        for word in queries[i].split():
            lab = query_tags.pop(0)
            word_list = tokenizer.tokenize(word)
            for w in word_list:
                l += [lab]
        l += ['[SEP]']
        labels += [l]
            

In [165]:
len(labels[0])

267

In [166]:
len(tokenized_texts[0])

267

In [154]:
max([len(t) for t in tokenized_texts])

395

In [167]:
max([len(t) for t in labels])

395

In [186]:
print(tokenized_texts[0])

['[CLS]', 'isolated', 'cr', '##anial', 'distortion', 'mimic', '##king', 'cap', '##ut', 'su', '##cc', '##ede', '##num', 'from', 'am', '##nio', '##tic', 'band', 'disruption', 'without', 'any', 'neurological', 'abnormal', '##ity', 'summary', 'this', 'report', 'describes', 'a', 'term', 'newborn', 'with', 'isolated', 'distortion', 'in', 'the', 'left', 'par', '##ie', '##tal', 'bone', 'without', 'any', 'other', 'visible', 'congenital', 'anomaly', ',', 'due', 'to', 'am', '##nio', '##tic', 'band', 'disruption', '.', 'a', 'skull', 'x', '-', 'ray', ',', 'ultrasound', 'scan', 'and', 'subsequent', 'mri', 'scan', 'of', 'the', 'brain', 'did', 'not', 'show', 'any', 'apparent', 'distortion', 'apart', 'from', 'depression', 'and', 'con', '##ca', '##vity', 'in', 'the', 'left', 'par', '##ie', '##tal', 'bone', '.', 'the', 'purpose', 'of', 'this', 'case', 'report', 'is', 'to', 'raise', 'awareness', 'of', 'this', 'possible', ',', 'mild', 'outcome', 'of', 'this', 'little', '-', 'known', 'entity', ',', 'which',

In [187]:
print(labels[0])

['[CLS]', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ans', 'B-ans', 'B-ans', 'I-ans', 'I-ans', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ans', 'B-ans', 'B-ans', 'I-ans', 'I-ans', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ans', 'B-ans', 'B-ans', 'I-ans', 'I-ans', 'I-ans', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [168]:
DATASET_SIZE = 53914
MAX_LEN = 424
SEED = 520
bs = 16

In [169]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([  101,  7275, 13675, 27532, 20870, 23150,  6834,  6178,  4904,
       10514,  9468, 14728, 19172,  2013,  2572, 27678,  4588,  2316,
       20461,  2302,  2151, 23130, 19470,  3012, 12654,  2023,  3189,
        5577,  1037,  2744, 20662,  2007,  7275, 20870,  1999,  1996,
        2187, 11968,  2666,  9080,  5923,  2302,  2151,  2060,  5710,
       27480, 28685,  1010,  2349,  2000,  2572, 27678,  4588,  2316,
       20461,  1012,  1037,  7412,  1060,  1011,  4097,  1010, 27312,
       13594,  1998,  4745, 27011, 13594,  1997,  1996,  4167,  2106,
        2025,  2265,  2151,  6835, 20870,  4237,  2013,  6245,  1998,
        9530,  3540, 24872,  1999,  1996,  2187, 11968,  2666,  9080,
        5923,  1012,  1996,  3800,  1997,  2023,  2553,  3189,  2003,
        2000,  5333,  7073,  1997,  2023,  2825,  1010, 10256,  9560,
        1997,  2023,  2210,  1011,  2124,  9178,  1010,  2029,  2089,
       23150,  6178,  4904, 10514,  9468, 11960,  2638,  2819,  1006,
        9587, 21285,

In [170]:
len(input_ids)

3771

In [171]:
len(input_ids[0])

424

In [172]:
token_type_ids = []
for ipid in input_ids:
    type_id = 0
    token_type_id = []
    for myid in ipid:
        token_type_id.append(type_id)
        if myid == 102:
            type_id+= 1
    token_type_ids.append(token_type_id)
    

In [173]:
print(token_type_ids[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [174]:
len(token_type_ids[0])

424

In [175]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["[PAD]"], padding="post",
                     dtype="long", truncating="post")


In [176]:
tags[0]

array([-100,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    0,    0,    0,    1,    1,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    0,    0,    0,    1,    1,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,   

In [177]:
len(tags)

3771

In [178]:
len(tags[0])

424

In [179]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [180]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=SEED, test_size=0.375)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=SEED, test_size=0.375)

In [181]:
tr_inputs = torch.tensor(tr_inputs)
tr_tags = torch.tensor(tr_tags)
tr_masks = torch.tensor(tr_masks)
val_inputs = torch.tensor(val_inputs)
val_tags = torch.tensor(val_tags)
val_masks = torch.tensor(val_masks)

In [37]:
torch.cuda.empty_cache() 

In [182]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [39]:
!nvidia-smi

Wed Mar 25 07:52:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   31C    P0    42W / 300W |     11MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [183]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))


In [41]:
model.cuda();

In [42]:
!nvidia-smi

Wed Mar 25 07:52:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   32C    P0    56W / 300W |   1472MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [184]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [185]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [45]:
epochs = 12
max_grad_norm = 1.0
F_scores = np.zeros(epochs,float)
patience = 6
for e in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    c_f1 = f1_score(pred_tags, valid_tags)
    print("F1-Score: {}".format(c_f1))
    #k epochs no improvement
    F_scores[e] = c_f1
    prev_Fs = np.arange(e-patience,e)
    prev_indices = prev_Fs[prev_Fs>=0]
    if all(F_scores[e] - F_scores[prev_indices] < 0 ) and e > patience:
        print('Done training')
        break
    else:
        print('Still training')
        #save checkpoint
        

Epoch:   0%|          | 0/12 [00:00<?, ?it/s]WARNING: Logging before flag parsing goes to stderr.
E0325 08:17:09.603682 47617041186496 ultratb.py:147] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/packages/7x/anaconda3/5.3.0/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-45-84f69950ef30>", line 18, in <module>
    loss.backward()
  File "/packages/7x/anaconda3/5.3.0/lib/python3.7/site-packages/torch/tensor.py", line 118, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/packages/7x/anaconda3/5.3.0/lib/python3.7/site-packages/torch/autograd/__init__.py", line 93, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/packages/7x/anaconda3/5.3.0/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_tr

KeyboardInterrupt: 

In [ ]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

In [ ]:
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME

output_dir = "./models/"

# Step 1: Save a model, configuration and vocabulary that you have fine-tuned

# If we have a distributed model, save only the encapsulated model
# (it was wrapped in PyTorch DistributedDataParallel or DataParallel)
model_to_save = model.module if hasattr(model, 'module') else model

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_dir)